# Загрузка данных

In [2]:
import pandas as pd

In [13]:
df = pd.read_csv('../data/train.csv').drop(columns=['index'])

In [14]:
df['class'] = pd.Categorical(df['class'])

In [15]:
df.head(3)

,text,class
0,Выплата на содержание родных детей,TAX
1,Оплата за Бритва Gillette Blue II Plus однораз...,NON_FOOD_GOODS
2,Паевой взнос в жилищно-строительный кооператив...,REALE_STATE


## Очистка датасета

In [2]:
!pip install natasha

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 6.8 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 18.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26095 sha256=57c634ce70af729a302fcb5381b2e8435fe06695622be212e234260f2c12dcc9
  Stored in directory: /home/misha/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built intervaltree

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from natasha import Doc, Segmenter, MorphVocab
from natasha import NewsEmbedding, NewsMorphTagger
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [27]:

# Инициализация компонентов Natasha
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

# Список месяцев
months = [
    "январь", "февраль", "март", "апрель", "май", "июнь",
    "июль", "август", "сентябрь", "октябрь", "ноябрь", "декабрь"
]

# Стоп-слова
russian_stopwords = set(stopwords.words("russian"))
russian_stopwords.discard('за')  # Убираем из стоп-слов "за"
russian_stopwords.update(['кг', 'сумма', 'тч', 'мл', 'счёт'])  # Добавляем свои слова

# Функция очистки текста
def clean_text(text, russian_stopwords, segmenter, morph_vocab, morph_tagger, months):
    # Удаляем даты из текста
    date_pattern = r"(\d{0,4}[./-]?)"
    cleaned_text = re.sub(date_pattern, "", text)
    
    # Токенизация и удаление стоп-слов
    tokens = word_tokenize(cleaned_text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords]
    tokens = [token for token in tokens if token not in string.punctuation]
    cleaned_text = ' '.join(tokens)
    
    # Используем Natasha для морфологической обработки
    doc = Doc(cleaned_text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    # Лемматизация
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    
    # Убираем месяцы и короткие слова
    lemmas = [token.lemma for token in doc.tokens if token.lemma not in months and len(token.lemma) > 1]
    
    return ' '.join(lemmas)

In [28]:
df['clean_text'] = df['text'].apply(clean_text)
df['clean_text']

0                        Выплата содержание родных детей
1          Оплата Бритва одноразовая шт счету июля Сумма
2      Паевой взнос жилищно строительный кооператив ц...
3      доставку д транспортом г Одинцово г Бийск дого...
4                     юридические услуги договору января
                             ...                        
395          Оплата Восточные сладости халва счету Сумма
396    транспортировку перевозка железной дороге г Бе...
397              Оплата дог Б г согл сч г счет фактура г
398        Оплата Масло оливковое жарки мл счету г Сумма
399    Оплата Крем универсальный Пантенол мл счету г ...
Name: clean_text, Length: 400, dtype: object

# Наивный байесовский классификатор

In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, f1_score

In [30]:
vectorizer = TfidfVectorizer()

### На грязном тексте

In [31]:
X = vectorizer.fit_transform(df.text.tolist())
y = df['class']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [34]:
y_pred = nb_model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1-Score: {f1_score(y_test, y_pred, average="macro")}')

Accuracy: 0.8375
F1-Score: 0.8051641372524225


### На чистом тексте

In [35]:
X = vectorizer.fit_transform(df.clean_text.tolist())
y = df['class']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [38]:
y_pred = nb_model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1-Score: {f1_score(y_test, y_pred, average="macro")}')

Accuracy: 0.875
F1-Score: 0.8397206139141624
